In [1]:
import pandas as pd
import re
import seaborn as sns
import os
import nltk
from nltk.stem.snowball import SnowballStemmer
# !pip install pymorphy2 
from nltk.corpus import stopwords
from functools import lru_cache
# from pymorphy2 import MorphAnalyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Здесь я минимально предобработал датасет, почистил пропущенные значения и почистил текст. Поскольку на лемматизацию ушло более 30 часов, после чего colab меня дисконектнул, я решил находить стеммы и уменьшить датасет до ≈ 600mb

In [2]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/archive.zip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.dropna(inplace=True)

In [6]:
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
739171,https://lenta.ru/news/2018/12/15/gasinparis/,«Желтые жилеты» прорвали кордон полиции под «М...,Протесты движения «желтых жилетов» в Париже сн...,Мир,Происшествия,2018/12/15
739172,https://lenta.ru/news/2018/12/15/su27/,Су-27 разбился на Украине,Истребитель Су-27 разбился в Житомирской облас...,Бывший СССР,Украина,2018/12/15
739173,https://lenta.ru/news/2018/12/15/spb/,В Петербурге нашли квартиру из 58 комнат,В Московском районе Санкт-Петербурга на продаж...,Дом,Квартира,2018/12/15
739174,https://lenta.ru/news/2018/12/15/gizin/,Падение горнолыжника на полной скорости попало...,Швейцарский горнолыжник Марк Гизин неудачно пр...,Спорт,Зимние виды,2018/12/15


In [7]:
def tokenize(text: str, regex="[А-Яа-яёA-z]+") -> list: # будем токенизировать с помощью регулярки
    regex = re.compile(regex)
    tokens = regex.findall(text.lower())

    return tokens

def remove_stopwords(
    lemmas, stopwords=stopwords.words("russian")):
    """Returns list of stemmas without stopwords"""
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

@lru_cache(maxsize=128) # ускорим стемматизацию кэшированием 
def stemmatize_word(token):
    """Returns stemma"""
    snowball = SnowballStemmer('russian')
    # pymorphy = MorphAnalyzer()
    # return pymorphy.parse(token)[0].normal_form
    return snowball.stem(token)

def stemmatize_text(text):
    """Returns list of stemmas"""
    return [stemmatize_word(w) for w in text]

def clean_text(text):
    """Returns list of stemmas without stopwords"""
    tokens = tokenize(text)
    lemmas = stemmatize_text(tokens)

    return remove_stopwords(lemmas)


In [8]:
from multiprocess import Pool # распараллелим процессы

def get_clean_text(df):
    with Pool(8) as p:
        stemmas = list(p.imap(clean_text, df))
    return stemmas


In [13]:
import numpy as np
np.random.seed(10)
# уменьшим датасет
remove_n = round(len(df) * 0.75)
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df = df.drop(drop_indices)

In [15]:
df.shape

(133623, 6)

In [16]:
df['stemmas'] = get_clean_text(df['title'] + ' ' + df['text'])

In [17]:
import ast

df['stemmas'] = df['stemmas'].apply(lambda x: ' '.join(x))

In [18]:
df.stemmas

4         люблин пойма швабск звер приеха варшав люблин ...
10        сентябр ввод форм таможен декларац сентябр тер...
15        дагеста идут активн боев действ сегодн район д...
17        коржак рассказа генпрокуратур финансов состоян...
19        киргиз ведет границ таджикистан узбекистан кир...
                                ...                        
739156    усик ответ угроз украинск националист абсолютн...
739158    париж протест арестова человек париж арестова ...
739160    прокомментирова решен объединительн собор киев...
739167    моурин сравн футболист мебел главн тренер манч...
739171    желт жилет прорва кордон полиц марсельез проте...
Name: stemmas, Length: 133623, dtype: object

In [19]:
df.to_csv('df_cleaned.csv', index=False)